In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import tensorflow_probability as tfp

import numpy as np

import GPyOpt

import argparse

from utils import tf_config, preprocess_data, search_algorithms, train

from models import fmri_ae, eeg_to_fmri, uniconv_fmri, bnn_fmri_ae

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

import time

dataset="01"
memory_limit=1700
n_individuals=8
interval_eeg=6

tf_config.set_seed(seed=42)
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

with tf.device('/CPU:0'):
    train_data, _ = preprocess_data.dataset(dataset, n_individuals=n_individuals, 
                                            interval_eeg=interval_eeg, 
                                            ind_volume_fit=False,
                                            standardize_fmri=False,
                                            iqr=True,
                                            verbose=True)
    _, fmri_train =train_data
    #eeg_val, fmri_val =val_data
    
    fmri_train = fmri_train[:296]
    
    train_x, val_x = train_test_split(fmri_train, test_size=0.2, shuffle=True, random_state=42)

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy
I: Starting to Load Data
I: Finished Loading Data
I: Pairs Created


In [2]:
batch_size=16
learning_rate=0.0001
skip_connections=True
maxpool=True
batch_norm=True
n_channels=8
latent_dimension=(5,5,5)
kernel_size=(5,5,3)
stride_size=(1,1,1)
n_stacks=5

optimizer = tf.keras.optimizers.Adam(learning_rate)

train_set = tf.data.Dataset.from_tensor_slices((train_x, train_x)).batch(batch_size)
dev_set = tf.data.Dataset.from_tensor_slices((val_x, val_x)).batch(1)

## Start playing with uncertainty

In [3]:
from models import bnn_fmri_ae

import importlib
importlib.reload(bnn_fmri_ae)

def create_bayesian_model(input_shape, output_shape, latent_shape, kernel_size, stride_size, n_channels,
                        maxpool=True, batch_norm=True, weight_decay=0.000, skip_connections=False,
                        n_stacks=2, local=True, local_attention=False, outfilter=0):
    
    model = bnn_fmri_ae.BNN_fMRI_AE(latent_shape, output_shape, 
                    kernel_size, stride_size, n_channels,
                    maxpool=maxpool, batch_norm=batch_norm,
                    skip_connections=skip_connections, n_stacks=n_stacks, 
                    local=True, local_attention=False, outfilter=0)
    
    model.build(input_shape=(None,)+train_x.shape[1:])
    
    return model


model = create_bayesian_model(train_x.shape[1:], train_x.shape[1:], latent_dimension,
                    kernel_size, stride_size, n_channels,
                    maxpool=maxpool, batch_norm=batch_norm,
                    skip_connections=skip_connections, n_stacks=n_stacks, 
                    local=True, local_attention=False, outfilter=0)

In [4]:
model.summary()

Model: "bnn_f_mri_ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           [(None, 64, 64, 30, 1), ( 31686591  
Total params: 31,686,591
Trainable params: 31,686,335
Non-trainable params: 256
_________________________________________________________________


In [5]:
def bnn_loss(y_true, y_pred):    
    D = y_true.shape[1]*y_true.shape[2]*y_true.shape[3]
    
    return (1/D)* tf.reduce_sum((tf.exp(-tf.math.log(y_pred[1]))*(y_pred[0] - y_true)**2)/2 + (tf.math.log(y_pred[1]))/2)


bnn_loss(fmri_train[0:1], model(fmri_train[0:1]))

InternalError: Blas GEMM launch failed : a.shape=(1, 1568), b.shape=(1568, 125), m=1, n=125, k=1568 [Op:MatMul]

In [ ]:
start_time = time.time()

#train
train_loss, val_loss = train.train(train_set, model, optimizer, 
                                   bnn_loss, epochs=10, 
                                   val_set=dev_set, verbose=True)

print("Time training: ", time.time()-start_time)